# ParFuMor (version YAML & Objets)

Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [13]:
# -*- coding: utf8 -*-
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
from cellbell import ding

In [14]:
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
annee="19"
numerosKalabas=[1,2,3,4,5]
numerosKalabas=[3]
nomKalabas=[repertoire+annee+"-K%d/"%num for num in numerosKalabas]

#nomKalabas=[home+"/Dropbox/Partage-Broadway/GAM/Caches/NosCaches/Kalabas/K2-Kalaba0304/"]

lexiqueTestPrep=u"dans"
lexiqueTestNoun=u"loups"
lexiqueTestHyper=u"villagEOIs"

In [15]:
PFM.duplicateErrors=[]
for serie in nomKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
    [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/19-K3/


/Users/gilles/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys
/Users/gilles/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  # This is added back by InteractiveShellApp.init_path()
/Users/gilles/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:17: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
/Users/gilles/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read 

VER : Temps Pers Genre
NOM : Genre Nombre Cas
DET : Genre Nombre Cas
ADJ : Genre Nombre
head stems
head stems,ADJ
head stems,DET
head stems,VER
head stems,NOM
head stems,NOM,A
head stems,NOM,C
head stems,NOM,B


In [16]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
print PFM.lexique.formeLexeme[u"enfANT"][0]
print PFM.lexique.formeLexeme[u"crocs"][0]

villageoise.B.N2
enfant.hyper.C.N3
croc.B.N1


In [17]:
paradigmes.getSigmas(["DET"])

[u'DET, Genre=A, Nombre=Sg, Cas=Nom',
 u'DET, Genre=A, Nombre=Sg, Cas=Acc',
 u'DET, Genre=A, Nombre=Sg, Cas=Dat',
 u'DET, Genre=A, Nombre=Sg, Cas=Obl',
 u'DET, Genre=A, Nombre=Du, Cas=Nom',
 u'DET, Genre=A, Nombre=Du, Cas=Acc',
 u'DET, Genre=A, Nombre=Du, Cas=Dat',
 u'DET, Genre=A, Nombre=Du, Cas=Obl',
 u'DET, Genre=A, Nombre=Pl, Cas=Nom',
 u'DET, Genre=A, Nombre=Pl, Cas=Acc',
 u'DET, Genre=A, Nombre=Pl, Cas=Dat',
 u'DET, Genre=A, Nombre=Pl, Cas=Obl',
 u'DET, Genre=B, Nombre=Sg, Cas=Nom',
 u'DET, Genre=B, Nombre=Sg, Cas=Acc',
 u'DET, Genre=B, Nombre=Sg, Cas=Dat',
 u'DET, Genre=B, Nombre=Sg, Cas=Obl',
 u'DET, Genre=B, Nombre=Du, Cas=Nom',
 u'DET, Genre=B, Nombre=Du, Cas=Acc',
 u'DET, Genre=B, Nombre=Du, Cas=Dat',
 u'DET, Genre=B, Nombre=Du, Cas=Obl',
 u'DET, Genre=B, Nombre=Pl, Cas=Nom',
 u'DET, Genre=B, Nombre=Pl, Cas=Acc',
 u'DET, Genre=B, Nombre=Pl, Cas=Dat',
 u'DET, Genre=B, Nombre=Pl, Cas=Obl',
 u'DET, Genre=C, Nombre=Sg, Cas=Nom',
 u'DET, Genre=C, Nombre=Sg, Cas=Acc',
 u'DET, Genr

In [18]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [19]:
regles.getRules("NOM",'CF=N3, Genre=C, Nombre=Du, Cas=Acc')

[]

In [20]:
test=Lexeme("RAD","DET","croc").getParadigm(lignes=["Genre"],colonne=["Nombre"])
test
#test.index.sortlevel(0,ascending=False)

del Cas


Nombre,Du,Pl,Sg
Genre,,,
A,"be-RAD-u-N,Zo-RAD-u-N,ka-RAD-u-N,si-RAD-u-N","ka-RAD-u-m,si-RAD-u-m,be-RAD-u-m,Zo-RAD-u-m","Zo-RAD-u-S,be-RAD-u-S,ka-RAD-u-S,si-RAD-u-S"
B,"si-RAD-i-N,be-RAD-i-N,Zo-RAD-i-N,ka-RAD-i-N","si-RAD-i-m,ka-RAD-i-m,Zo-RAD-i-m,be-RAD-i-m","be-RAD-i-S,ka-RAD-i-S,Zo-RAD-i-S,si-RAD-i-S"
C,"be-RAD-e-N,si-RAD-e-N,ka-RAD-e-N,Zo-RAD-e-N","ka-RAD-e-m,si-RAD-e-m,be-RAD-e-m,Zo-RAD-e-m","si-RAD-e-S,be-RAD-e-S,ka-RAD-e-S,Zo-RAD-e-S"


In [21]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{}

In [22]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

poisson.A.N2 geZaj
main.B.N1 tajen
courageux.A2 TeNum
envahir.V2 najok
transformer.V2 parap
mort.B.N2 nonom
trois.A1 deriNa
sortir.V2 logar
acheter.V1 buliza
blessure.A.N3 nalus
chercher.V1 kifir
donner.V1 kusiT
souris.C.N3 somal
viande.A.N3 koZoN
sort.C.N1 lojit
lune.B.N3 nuriT
être.V1 dalut
visage.A.N2 zakem
œuf.C.N1 gotar
grand.A1 Zojem
ENTRE but
villageois.C.N3 pujaS
apporter.V1 bawek
jaune.A1 zamoN
de tow
infirmière.A.N1 Torol
louve.B.N2 nimaru
souffrance.C.N2 paZer
villageoise.B.N2 Nikol
profond.A1 nadir
suivant.A2 Tajun
disparaître.V2 voniS
IND.ART k
corps.C.N1 muZur
thé.C.N3 noDep
furieux.A1 surezi
gros.A2 rakek
coyote.A.N2 jurin
rêvasser.V1 maber
fuir.V1 luram
quatre.A2 rujem
pour ruS
blessé.A1 rebin
boire.V2 mename
arriver.V2 samoj
chambre.C.N1 lebaS
Mahira.C.N1 maira
aller.V2 puger
Nicole.C.N2 nikol
sur Sol
enfant.hyper.C.N3 keZuNo
milieu.B.N1 lenoma
jeter.V1 bawuN
dévorer.V1 geget
Violette.B.N1 vjolet
parler.V1 Sogap
bas.A2 tarona
sauter.V2 mulur
soeur.C.N3 susen
nuit.B.N3 

In [23]:
ding()